In [ ]:
import os
import requests

import folium
import geopandas as gpd
import pandas as pd
from datetime import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pystac_client import Client
from shapely.geometry import shape, box, mapping


def get_aviris_data(collection):
    """
    Get all AVIRIS data from a specific collection

    Parameters
    ----------
    collection : str
        The collection to query from cmr.earthdata.nasa.gov

    Returns
    -------
    List
        A list of granules / objects
    """

    url = "https://cmr.earthdata.nasa.gov/search/granules.json"
    params = {
        "collection_concept_id": collection,  # "C2659129205-ORNL_CLOUD",
        "page_size": 1000,
        "page_num": 1,
        "sort_key": "-start_date",
        "temporal": "2022-07-01T00:00:00.000Z,",
    }

    response = requests.get(url, params=params)

    # Raise an error if something went wrong
    response.raise_for_status()

    # Parse JSON into a Python dictionary
    data = response.json()

    # Example: print how many granules you got
    granules = len(data.get("feed", {}).get("entry", []))
    print(f"Got {granules} granules, aka {granules // 2} images (.bin + .hdr files)")

    # Access the whole dictionary if you want
    granules = data["feed"]["entry"]
    granules = sorted(granules, key=lambda x: x["title"])

    return granules


def get_enmap_images(aoi, start_date, end_date, cloud_cover_max=50):
    """
    Query and plot EnMAP L2A images from the DLR STAC API.

    Parameters
    ----------
    aoi : dict or list[float]
        Area of interest, either a GeoJSON Polygon or bounding box [minx, miny, maxx, maxy].
    start_date : str
        Start of date range, e.g. "2025-01-01".
    end_date : str
        End of date range, e.g. "2025-11-01".
    cloud_cover_max : float
        Maximum allowed cloud coverage (%). Default 50.

    Returns
    -------
    geopandas.GeoDataFrame
        containing all EnMAP images that match the filters
    """

    # Convert bounding box to GeoJSON polygon if needed
    if isinstance(aoi, list) and len(aoi) == 4:
        geom = box(*aoi)
    else:
        geom = shape(aoi)

    # STAC API endpoint
    stac_url = "https://geoservice.dlr.de/eoc/ogc/stac/v1"

    # Open STAC client
    client = Client.open(stac_url)

    # Search collection
    search = client.search(
        collections=["ENMAP_HSI_L2A"],
        intersects=mapping(geom),
        datetime=f"{start_date}/{end_date}",
        query={"eo:cloud_cover": {"lt": cloud_cover_max}},
        limit=200,
    )

    # Convert results to GeoDataFrame
    items = list(search.get_items())
    if not items:
        print("No matching scenes found.")
        return None
    print(f"Found {len(items)} scenes")

    # Extract just geometries + key properties for plotting
    plot_features = []
    for item in items:
        props = item.properties.copy()
        props["id"] = item.id
        props["cloud_cover"] = props.get("eo:cloud_cover")
        props["href_self"] = item.get_self_href()
        props["asset_keys"] = list(item.assets.keys())
        props["image"] = item.assets.get("image")
        plot_features.append(
            {"type": "Feature", "geometry": item.geometry, "properties": props}
        )

    gdf = gpd.GeoDataFrame.from_features(plot_features, crs="EPSG:4326")
    gdf = gdf[gdf["eo:cloud_cover"].astype(int) < cloud_cover_max]
    print(f"{len(gdf)} scenes have less than {cloud_cover_max} cloud coverage ...")

    return gdf


In [ ]:
aviris_granules = get_aviris_data("C2659129205-ORNL_CLOUD")

In [ ]:
def is_below_45_deg(raw_geom):
    # split into floats
    vals = list(map(float, raw_geom.split()))
    
    # group into coordinate pairs (lat, lon → lon, lat)
    coords = [[vals[i+1], vals[i]] for i in range(0, len(vals), 2)]  # lat, lon pairs

    return coords[0][1] < 45

aviris_granules_filtered = list(filter(lambda x: is_below_45_deg(x['polygons'][0][0]), aviris_granules))
print(f"Selecting only those with latitude < 45: length now: {len(aviris_granules_filtered)}")

In [ ]:
def download_enmap_file(url, fname):           
    print(f"⬇️ Downloading {fname} from {url} ...")
    SESSION = "_q9f8voUZmHEwCNnFUZDxQ|1762983559|FKCFV_PU6EWqo7R3I3tLIOPtCGqJ3kT46cwm0JnUBr0cHksiVvlqc9m-zcj7gYTzCF6SPTnUOfr1XDrGvE3A_JlURmzcsBKv_2BaSK-N-u-MW9Tp1gk4haPKXiSutL2tfHRiMoFFNUnEZJHb57ErFgQoQLIV4heNx1oDO6MBdFSQpOK2A3DjFImH68bMAZ2mN5T2IrP3YsE_N1OXXZl-9iatnEdFUOfzVTQPtmDQUnQ|zf1q0sSHaG_IKzSh9mT-uDEicI4"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:144.0) Gecko/20100101 Firefox/144.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Connection": "keep-alive",
        f"Cookie": "session={SESSION}",
    }
    
    # Stream to avoid loading large file in memory
    with requests.get(url, headers=headers, stream=True, timeout=600) as r:
        if r.status_code == 200 and "text/html" not in r.headers.get("Content-Type", ""):
            with open(fname, "wb") as f:
                for chunk in r.iter_content(chunk_size=16384):
                    f.write(chunk)
            print("✅ Download complete")
        else:
            print("⚠️ Something went wrong:", r.status_code)
            print("Returned URL:", r.url)
            print("Response headers:", r.headers)


def download_aviris_file(url, fname):           
    print(f"⬇️ Downloading {fname} from {url} ...")
    ASF_URS = ""
    ACCESS_TOKEN = ""
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:144.0) Gecko/20100101 Firefox/144.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Connection": "keep-alive",
        "Cookie": (
            "urs_user_already_logged=yes; "
            "urs_guid_ops=03a1a0ea-4bc0-4490-b199-6b26c40bc995; "
            f"asf-urs={ASF_URS}; "
            f"accessToken={ACCESS_TOKEN}"
        ),
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Priority": "u=0, i",
        "TE": "trailers"
    }
    
    # Stream to avoid loading large file in memory
    with requests.get(url, headers=headers, stream=True, timeout=600) as r:
        if r.status_code == 200 and "text/html" not in r.headers.get("Content-Type", ""):
            with open(fname, "wb") as f:
                for chunk in r.iter_content(chunk_size=16384):
                    f.write(chunk)
            print("✅ Download complete")
        else:
            print("⚠️ Something went wrong:", r.status_code)
            print("Returned URL:", r.url)
            print("Response headers:", r.headers)


BASE_DOWNLOAD_DIR = "."

for g in aviris_granules_filtered[::2]:
    print(g['title'])
    ########### First, do geometry ##########
    
    raw_geom = g['polygons'][0][0]
    # g['polygons'] is like [['66.4241714 -147.238739 66.4497986 -147.2604065 66.3321457 -147.9172363 66.3065262 -147.8955231 66.4241714 -147.238739']]
    
    # split into floats
    vals = list(map(float, raw_geom.split()))
    
    # group into coordinate pairs (lat, lon → lon, lat)
    coords = [[vals[i+1], vals[i]] for i in range(0, len(vals), 2)]  # lat, lon pairs
    print(coords)
    
    # turn into a GeoJSON polygon
    aoi = {
        "type": "Polygon",
        "coordinates": [coords]
    }

    ########### Then, -1 to +1 month ##########
    time_start_str = g['time_start']
    
    # Convert string to datetime object
    time_start = datetime.strptime(time_start_str, "%Y-%m-%dT%H:%M:%S.%fZ").date()
    
    # Compute start and end dates ±1 month
    start_date = time_start - relativedelta(months=1)
    end_date   = time_start + relativedelta(months=1)
    
    gdf = get_enmap_images(aoi, start_date, end_date, cloud_cover_max=30)
    if gdf is not None:
        # First, drop duplicates with old version number 
        
        # Make sure datetime columns are parsed
        gdf['start_datetime'] = pd.to_datetime(gdf['start_datetime'])
        gdf['version'] = gdf['version'].astype(str)
        
        # Sort by start_datetime (ascending) and version (descending)
        # Assumes version strings like "01.05.02" compare correctly lexicographically
        gdf_sorted = gdf.sort_values(
            by=['start_datetime', 'version'],
            ascending=[True, False]
        )
        
        # Drop the duplicates with old version now
        gdf_unique = gdf_sorted.drop_duplicates(subset='start_datetime', keep='first')

        # Let's download stuff
        # AVIRIS first
        # Links are both https and S3
        img_bin_links = sorted([link['href'] for link in g['links'] if link['href'].endswith('_img.bin')])
        # sorted, https:// comes before s3://
        download_aviris_file(img_bin_links[0], os.path.join(BASE_DOWNLOAD_DIR, g['title']))
        
        # Download all EnMAPimages from gdf_unique 
        for _, row in gdf_unique.iterrows():
            url = row["image"].href
            download_dir = os.path.join(BASE_DOWNLOAD_DIR, os.path.splitext(g['title'])[0])
            os.path.makedirs(download_dir, exist_ok=True)
            fname = os.path.join(download_dir, f"{row.id}.tif")
            if not os.path.exists(fname):
                download_enmap_file(url, fname)